In [ ]:
!pip install --upgrade sentence-transformers langchain langchain_community langchain_huggingface faiss-cpu langchain_google_vertexai langchainhub langgraph httpx

In [ ]:
import os
from langchain_community.tools.tavily_search import TavilySearchResults
from langgraph.prebuilt import create_react_agent
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings
from langchain import PromptTemplate
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.tools.retriever import create_retriever_tool
from langchain.agents import AgentExecutor, create_openai_tools_agent
from langchain_core.messages import HumanMessage
from langchain_google_vertexai import ChatVertexAI
from google.colab import auth
import vertexai
from langgraph.prebuilt import create_react_agent

In [ ]:
os.environ['TAVILY_API_KEY'] = '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'

In [ ]:
auth.authenticate_user()

In [ ]:
vertexai.init(project='mercurial-smile-425913-v3')

In [ ]:
model = ChatVertexAI(model="gemini-pro")

In [ ]:
model.invoke('Write the summary of 2024 UCL final')

AIMessage(content='As of 26 October 2023, the 2024 UEFA Champions League final has not yet been played. Therefore, I cannot provide a summary of the match. The final is scheduled to take place on Saturday, 1 June 2024, at Wembley Stadium in London, England. The two teams that will contest the final will be determined through a two-legged knockout tournament that begins in September 2023.\n\nWould you like me to provide you with information about the teams that are currently participating in the 2023-2024 UEFA Champions League? Or perhaps you would like me to remind you about the results of the 2023 UEFA Champions League final?', response_metadata={'is_blocked': False, 'safety_ratings': [{'category': 'HARM_CATEGORY_HATE_SPEECH', 'probability_label': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_DANGEROUS_CONTENT', 'probability_label': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_HARASSMENT', 'probability_label': 'NEGLIGIBLE', 'blocked': False}, {'category'

In [ ]:
search_tool = TavilySearchResults(max_results=1)

In [ ]:
search_tool.invoke("Write the summary of 2024 UCL final")

[{'url': 'https://www.sportingnews.com/in/football/news/who-won-champions-league-final-2024-real-madrid-borussia-dortmund/c2aa2d0b8666ccb0125474be',
  'content': "Real Madrid came through a torrid first half against Borussia Dortmund to win the UEFA Champions League final 2-0 at Wembley Stadium. Edin Terzic's underdogs made the running during a first half ..."}]

In [ ]:
embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when p

In [ ]:
loader = WebBaseLoader("https://docs.smith.langchain.com/overview")

In [ ]:
docs = loader.load()

In [ ]:
documents = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200).split_documents(docs)

In [ ]:
vector = FAISS.from_documents(documents, embeddings)

In [ ]:
retriever = vector.as_retriever()

In [ ]:
retriever_tool = create_retriever_tool(
  retriever,
  "langsmith_search",
  "Search for information about LangSmith. For any questions about LangSmith, you must use this tool!",
)

In [ ]:
tools = [search_tool, retriever_tool]

In [ ]:
model_with_tools = model.bind_tools(tools)

In [ ]:
response = model_with_tools.invoke([HumanMessage(content="Write the summary of 2024 UCL final search")])
print(f"ContentString: {response.content}")
print(f"ToolCalls: {response.tool_calls}")

ContentString: 
ToolCalls: [{'name': 'tavily_search_results_json', 'args': {'query': '2024 UCL final summary'}, 'id': '8c819844-e9ce-4f67-b113-2a0c1b14c0a5'}]


In [ ]:
agent_executor = create_react_agent(model, tools)

In [ ]:
response = agent_executor.invoke({"messages": [HumanMessage(content="use tavily Write the summary of 2024 UCL final")]})

In [ ]:
response["messages"]

[HumanMessage(content='use tavily Write the summary of 2024 UCL final', id='7d6f8ab1-7342-4a77-a57d-802e87c15601'),
 AIMessage(content='', additional_kwargs={'function_call': {'name': 'tavily_search_results_json', 'arguments': '{"query": "2024 UCL final"}'}}, response_metadata={'is_blocked': False, 'safety_ratings': [{'category': 'HARM_CATEGORY_HATE_SPEECH', 'probability_label': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_DANGEROUS_CONTENT', 'probability_label': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_HARASSMENT', 'probability_label': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_SEXUALLY_EXPLICIT', 'probability_label': 'NEGLIGIBLE', 'blocked': False}], 'usage_metadata': {'prompt_token_count': 105, 'candidates_token_count': 15, 'total_token_count': 120}}, id='run-5bc322e6-97ba-47a6-9e97-64985dcefd50-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': '2024 UCL final'}, 'id': '899b68f0-db04-4b6c-a233-96d2890377fc'